In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [3]:
from time import sleep

from ICARUS.database import DB
import matplotlib.pyplot as plt
import numpy as np

In [8]:
airfoil = DB.get_airfoil('NACA0009')

fig = plt.figure()
ax = fig.add_subplot(111)

# airfoil.plot(ax=ax)
airfoil_flap = airfoil.flap2(
    flap_hinge= 0.63,
    flap_angle= 10.,
    chord_extension= 1., 
)

# airfoil_flap.plot(ax=ax, overide_color = 'green', scatter = True)
airfoil_flap.repanel_from_internal(120, "cosine")
airfoil_flap.plot(ax=ax, overide_color = 'red')
airfoil_flap.repanel_spl(120, 0)
airfoil_flap.plot(ax=ax, overide_color = 'blue')

In [5]:
from ICARUS.core.units import calc_reynolds
import numpy as np
from ICARUS.core.types import FloatArray

# PARAMETERS FOR ESTIMATION
chord_max: float = 0.5
chord_min: float = 0.1
u_max: float = 35.0
u_min: float = 5.0
viscosity: float = 1.56e-5
speed_of_sound: float = 340.3

# MACH ESTIMATION
mach_max: float = 0.0
# mach_min: float = calc_mach(10, speed_of_sound)
# mach: FloatArray = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max

# REYNOLDS ESTIMATION
RE_MIN = 8e4
RE_MAX = 1.5e6
NUM_BINS = 12
REYNOLDS_BINS = np.logspace(-2.2, 0, NUM_BINS) * (RE_MAX - RE_MIN) + RE_MIN
reynolds = REYNOLDS_BINS

# ANGLE OF ATTACK SETUP
aoa_min: float = -10
aoa_max: float = 16
num_of_angles: int = int((aoa_max - aoa_min) * 2 + 1)
angles: FloatArray = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)

# Transition to turbulent Boundary Layer
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}
ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
Ncrit = 9

In [6]:
from ICARUS.computation.solvers.solver import Solver
# # Xfoil
from ICARUS.computation.solvers.Xfoil.xfoil import Xfoil

print(f"\nRunning airfoil {airfoil}\n")

for flap_angle in np.arange(7.5, 35, 2.5):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    airfoil_flap = airfoil.flap2(
        flap_hinge= 0.63,
        flap_angle=flap_angle,
        chord_extension= 1., 
    )
    airfoil_flap.repanel_from_internal(120, "cosine")
    airfoil_flap.repanel_spl(120, 1e-6)

    airfoil_flap.plot(scatter=True, ax = ax)
    airfoil_flap.plot( ax = ax)
    plt.show(block=True)
    xfoil: Solver = Xfoil()

    # Import Analysis
    analysis: str = xfoil.get_analyses_names()[1]  # Run
    xfoil.select_analysis(analysis)

    # Get Options
    options = xfoil.get_analysis_options(verbose=False)
    solver_parameters = xfoil.get_solver_parameters()

    # Set Options
    options.airfoil = airfoil_flap
    options.mach = MACH
    options.reynolds = reynolds
    options.min_aoa = aoa_min
    options.max_aoa = aoa_max
    options.aoa_step = 0.5

    # Set Solver Options
    solver_parameters.max_iter = 200
    solver_parameters.Ncrit = 9
    solver_parameters.xtr = (.05, .05)
    solver_parameters.print = False
    solver_parameters.repanel_n = 120

    xfoil.define_analysis(options, solver_parameters)
    xfoil.print_analysis_options()

    # RUN
    xfoil.execute(parallel=False)
    # Get polar
    polar = DB.foils_db.get_polars(airfoil_flap.name.upper())
    fig = polar.plot()
    fig.show()
    plt.show(block=True)



Running airfoil Airfoil(upper, lower)

Airfoil Polar Analysis For a multiple Reynolds using aseq
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds using aseq: 

| VarName   | Value                                                          | Description                       |
|-----------|----------------------------------------------------------------|-----------------------------------|
| airfoil   | Complex Datatype (naca0009_flapped_hinge_0.63_deflection_7.50) | Airfoil Object                    |
| mach      | 0.0                                                            | Mach number                       |
| reynolds  | Complex Datatype                                               | List of Reynold's numbers to run  |
| min_aoa   | -10                                                            | Minimum angle of attack           |
| max_aoa   | 16                                                             | Maximum angle of attack           |
| aoa

  0%|          | 0/12 [00:00<?, ?it/s]

Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Adding NACA0009_FLAPPED_HINGE_0.63_DEFLECTION_7.50 to the database
Analysis Completed


c:\Users\tryfo\anaconda3\envs\aero\Lib\site-packages\scipy\interpolate\_interpolate.py:710: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\tryfo\anaconda3\envs\aero\Lib\site-packages\numpy\lib\function_base.py:1238: RuntimeWarning: divide by zero encountered in divide
  out[tuple(slice1)] = (f[tuple(slice4)] - f[tuple(slice2)]) / (2. * ax_dx)
c:\Users\tryfo\anaconda3\envs\aero\Lib\site-packages\numpy\lib\function_base.py:1259: RuntimeWarning: divide by zero encountered in scalar divide
  out[tuple(slice1)] = (f[tuple(slice2)] - f[tuple(slice3)]) / dx_0
c:\Users\tryfo\anaconda3\envs\aero\Lib\site-packages\numpy\lib\function_base.py:1266: RuntimeWarning: divide by zero encountered in scalar divide
  out[tuple(slice1)] = (f[tuple(slice2)] - f[tuple(slice3)]) / dx_n


Airfoil Polar Analysis For a multiple Reynolds using aseq
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds using aseq: 

| VarName   | Value                                                           | Description                       |
|-----------|-----------------------------------------------------------------|-----------------------------------|
| airfoil   | Complex Datatype (naca0009_flapped_hinge_0.63_deflection_10.00) | Airfoil Object                    |
| mach      | 0.0                                                             | Mach number                       |
| reynolds  | Complex Datatype                                                | List of Reynold's numbers to run  |
| min_aoa   | -10                                                             | Minimum angle of attack           |
| max_aoa   | 16                                                              | Maximum angle of attack           |
| aoa_step  | 0.5                     

  0%|          | 0/12 [00:00<?, ?it/s]

Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Repaneling Airfoil with 120
Adding NACA0009_FLAPPED_HINGE_0.63_DEFLECTION_10.00 to the database
Analysis Completed


PolarsNotFoundError: Polars for airfoil NACA0009_FLAPPED_HINGE_0.63_DEFLECTION_10.00 not found in database for solver None! The available solvers are ['None']

In [ ]:
airfs = []
for flap_angle in np.arange(0, 35, 2.5):
    try:
        airfoil_flap = airfoil.flap(
            flap_hinge= 0.63,
            flap_angle=flap_angle,
            chord_extension= 1., 
        )
        airfs.append(airfoil_flap)
        polar = DB.foils_db.get_polars(airfoil_flap.name.upper())
        fig = polar.plot()
        fig.show()
    except:
        pass

airf_names = [airf.name for airf in airfs]

In [ ]:
airf_names

In [ ]:
from ICARUS.visualization.airfoil.db_polars import plot_airfoils_polars

plot_airfoils_polars(
    airf_names[1:],
    reynolds= 4e5,
    solvers=["Xfoil"]
)